### 네이버 API를 이용한 지식인 정보 검색
- 네이버 개발자센터에 가입 및 인증서 발급

In [28]:
import os
import sys
import json
import urllib.request
import requests
import pandas as pd

### urllib.request

In [9]:
client_id = ""
client_secret = ""

encText = urllib.parse.quote("제주도")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # json 결과
#url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # xml 결과
    
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)

rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
else:
    print("Error Code:" + rescode)

In [16]:
res_json=json.loads(response_body)
res_json["total"]

1145852

### requests

In [18]:
encText = urllib.parse.quote("제주도")

url = "https://openapi.naver.com/v1/search/kin?query=" + encText # json 결과
#url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # xml 결과
    
headers = {"X-Naver-Client-Id":"TWoFiP4PRGZBGB6H_JSm", 
           "X-Naver-Client-Secret":"TOXDluSLJh"}

res_json=requests.get(url, headers=headers)

if res_json.status_code == 200:
    res_content = res_json.json()
else:
        print("Error Code:" + str(res_json.status_code))
        sys.exit(0)
        
res_content["total"]

1145861

### 세부 정보를 위한 url 요청

In [24]:
# 데이터의 key 값 확인
print(res_content.keys())
print(res_content['items'][0].keys())

dict_keys(['lastBuildDate', 'total', 'start', 'display', 'items'])
dict_keys(['title', 'link', 'description'])


In [25]:
# 각각의 데이터 확인
print(res_content['items'][0]['title'])
print()
print(res_content['items'][0]['link'])
print()
print(res_content['items'][0]['description'])

<b>제주도</b> 신혼여행 질문입니다.

https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90112&docId=413278778&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0

... 신혼여행을 5월1일날 <b>제주도</b>로 가게되었는데 그때의 <b>제주도</b> 날씨는 많이 더울까요? 꽃은 어떤 것들이 많이... 질문하신 [<b>제주도</b> 날씨] 관련 답변 드리도록 하겠습니다. 5월에 <b>제주도</b>로 신혼여행을 떠나시는군요? 3월에서 5월까지... 


In [27]:
# url 정보 전체 출력
url_lst = []
for item in res_content['items']:
    url_lst.append(item['link'])
url_lst

['https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90112&docId=413278778&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=1&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=9&dirId=90112&docId=412188026&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=2&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=9&dirId=9011401&docId=412670660&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=3&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=9&dirId=9020602&docId=413295158&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=4&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=1214&docId=411913385&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=5&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=12&dirId=121602&docId=410479221&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&rank=6&search_sort=0&spq=0',
 'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=81302&docId=414096108&qb=7KCc7KO864+E&enc=utf8&section=kin.qna&r

### 가져온 정보에서 제목/링크/요약를 DataFrame로 저장

In [31]:
df=pd.DataFrame(res_content['items'])
df.head(2)

,title,link,description
0,<b>제주도</b> 신혼여행 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=9&...,... 신혼여행을 5월1일날 <b>제주도</b>로 가게되었는데 그때의 <b>제주도<...
1,<b>제주도</b> 3박4일 여행코스,https://kin.naver.com/qna/detail.naver?d1id=9&...,2월 26일 3박4일로 여자친구랑 <b>제주도</b> 여행을 가려고 하는데 <b>제...


## 네이버 API에서 20개 정보 가져와 질문정보/답변정보 저장하기
- txt 형식으로 저장(키워드_질문.txt, 키워드_답변.txt)

In [59]:
import os
import sys
import json
import urllib.request
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re

In [64]:
# 네이버 API 로그인 후 link 정보 가져와 돌려주기
def get_Nav_client(encText, sPage):
    url = "https://openapi.naver.com/v1/search/kin?query=" + encText # json 결과
    url = url + "&start=" + str(sPage) + "&display=20"

    headers = {"X-Naver-Client-Id":"TWoFiP4PRGZBGB6H_JSm", 
               "X-Naver-Client-Secret":"TOXDluSLJh"}

    res_content=requests.get(url, headers=headers)

    if res_content.status_code == 200:
        res_json = res_content.json()
    else:
        print("Error Code:" + str(res_json.status_code))
        sys.exit(0)
    
    df=pd.DataFrame(res_json['items'])
    
    return list(df['link'])

In [68]:
#link를 이용해 세부 내용 가져와 파일에 기록하기
def get_write(url):
    kin_html = requests.get(url)

    fq = open('c:/pydata/제주도_질문.txt', "a")
    fa = open('c:/pydata/제주도_답변.txt', "a")

    if kin_html.status_code != 200 :
        print("Error Code:" + str(res_json.status_code))
        sys.exit("데이터가 없어 강제 종료합니다.")

    kin_soup = bs(kin_html.text, "html.parser")

    # 제목 가져오기
    try:
        kin_tit = kin_soup.find("div", class_="title").get_text().replace("\n", "").replace("\t", "")
        fq.write(kin_tit+"\n")
    except:
        fq.write("\n")

    kin_body=kin_soup.find('div', class_="c-heading__content")

    # 질문 내용
    try:
        kin_body=kin_soup.find('div', class_="c-heading__content").text.replace("\n", "").replace("\t", "")
        fq.write(kin_body)
    except:
        fq.write("\n")

    # 답변 내용

    for kin_ans in kin_soup.find_all('div', class_="se-component se-text se-l-default"):
        fa.write(re.sub("[^가-힣 ]","",kin_ans.text))
    fq.write("\n")

    fq.close()
    fa.close()

In [71]:
encText = urllib.parse.quote("제주도")
reg_link=get_Nav_client(encText, 1)

# 기록할 빈 파일 생성
fq = open('c:/pydata/제주도_질문.txt', "w")
fa = open('c:/pydata/제주도_답변.txt', "w")
fq.close()
fa.close()

# link를 이용해 세부 내용 가져와 파일에 기록하기
for link in reg_link:
    get_write(link)

print("=== End ===")

=== End ===


### [미션1]  데이터 1000개를 가져와 텍스트 파일로 저장하기
- 질문/답변으로 나누어 저장하기
- 한번에 가져올수 있는 데이터는 100개

### [미션2] 데이터 1000개를 가져와 DataFrame으로 저장하기
- 제목/질문내용/답변내용으로 구분해 저장
- 답변의 1개의 질문에 대하여 전체 답변을 묶어서 저장(개인별로는 "\n"로 구분해 하나의 index에 저장)
- df.loc[0] => 1개 제목/ 1개 질문 / 질문에 대한 답변 전체